In [1]:
print('test')

test


In [2]:
# Bulk import cell
import wandb
import random
import torch
import transformers
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from vit_pytorch.mobile_vit import MobileViT
import torchvision
from torchvision import transforms
from torch.nn import functional as F
from x_transformers import ViTransformerWrapper, Encoder




In [3]:
platform = 'm1'
# platform = '2015'



if platform == '2015':
  dataset_path = '/Users/rotemisraeli/Documents/datasets/TinyImageNet/'
elif platform == 'm1':
  dataset_path = '/Volumes/black_ssd/datasets/imagenet/imagenet-object-localization-challenge/ILSVRC/Data/images/'

pl.seed_everything(1234)

Global seed set to 1234


1234

In [4]:
wandb.login()

wandb: Currently logged in as: rotem98 (use `wandb login --relogin` to force relogin)


True

In [5]:
project = "mobile_test2"  # W&B project name here
entity = 'rotem98'  # your W&B username or teamname here
img_size = 192

In [6]:
class SentenceClassifier(pl.LightningModule):

  def __init__(self, learning_rate=5e-5):
    super(SentenceClassifier, self).__init__()
    # self.model = MobileViT(
    #   image_size = (64, 64),
    #   dims = [96, 120, 144],
    #   channels = [16, 32, 48, 48, 64, 64, 80, 80, 96, 96, 384],
    #   num_classes = 100
    # )
    self.model = ViTransformerWrapper(
      image_size = img_size,
      patch_size = 16,
      num_classes = 1000,
      attn_layers = Encoder(
          dim = 256,
          depth = 4,
          heads = 4,
          use_qk_norm_attn = True, # set this to True
          qk_norm_attn_seq_len = 256 # set this to max_seq_len from above
      )
    )
    self.learning_rate = learning_rate

  def training_step(self, batch, batch_no):
    input, labels = batch
    outputs = self.model(input)
    preds = torch.argmax(outputs, axis=1)
    correct = sum(preds.flatten() == labels.flatten())
    loss = F.cross_entropy(outputs, labels)
    self.log("train/loss", loss, on_step=True, on_epoch=True)
    self.log("train/acc", correct/len(batch), on_step=True, on_epoch=True)
    return loss

  def validation_step(self, batch, batch_no):
    input, labels = batch
    outputs = self.model(input)
    preds = torch.argmax(outputs, axis=1)
    correct = sum(preds.flatten() == labels.flatten())
    loss = F.cross_entropy(outputs, labels)
    self.log("val/loss", loss, on_step=False, on_epoch=True)
    self.log("val/acc", correct/len(batch), on_step=False, on_epoch=True)

  def configure_optimizers(self):
    return transformers.AdamW(
        self.model.parameters(),
        lr = self.learning_rate,
        eps = 1e-8
    )

def data_pre(config):
    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(img_size),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225] )
    ])
    val_transform = transforms.Compose([
      transforms.Resize(img_size),
      transforms.CenterCrop(img_size),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225] )
    ])
    if platform == 'm1':
        dataset = torchvision.datasets.ImageFolder(root=dataset_path+'train', transform=val_transform)
        size1 = int(len(dataset)*0.45)
        size2 = int(len(dataset)*0.05)
        size3 = len(dataset) - size2 - size1
        train_set, val_set,not_use_set = torch.utils.data.random_split(dataset,[size1,size2,size3])
        train_data_loader = torch.utils.data.DataLoader(train_set, batch_size=config.batch_size, shuffle=True,  num_workers=4)
        val_data_loader = torch.utils.data.DataLoader(val_set, batch_size=config.batch_size, shuffle=False,  num_workers=4)
    elif platform == '2015':
        train_data = torchvision.datasets.ImageFolder(root=dataset_path+'train', transform=train_transform)
        train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=config.batch_size, shuffle=True,  num_workers=4)
        val_data = torchvision.datasets.ImageFolder(root=dataset_path+'val', transform=val_transform)
        val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=config.batch_size, shuffle=False,  num_workers=4)

    return train_data_loader,val_data_loader


In [7]:
def train(config):
    with wandb.init(project=project, entity=entity, job_type="train", config=config,dir='wandb_dir') as run:
        config = run.config

        train_data_loader, val_data_loader = data_pre(config)

        model = SentenceClassifier(learning_rate=config.learning_rate)

        logger = pl.loggers.WandbLogger(experiment=run, log_model=True,save_dir='wandb_logger_savedir')

        gpus = -1 if torch.cuda.is_available() else 0

        trainer = pl.Trainer(max_epochs=config.epochs, gpus=gpus, logger=logger,weights_save_path='models')

        trainer.fit(model, train_data_loader, val_data_loader)

In [ ]:
train(config={"learning_rate": 5e-5, "batch_size": 64, "epochs": 10})

wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory.
wand

wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory.


wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory.
/Users/rotemisraeli/opt/anaconda3/envs/python/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:57: LightningDeprecationWarning: Setting `Trainer(weights_save_path=)` has been deprecated in v1.6 and will be removed in v1.8. Please pass ``dirpath`` directly to the `ModelCheckpoint` callback
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/rotemisraeli/opt/anaconda3/envs/python/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name  | Type                 | Params
-------------------------------

Sanity Checking: 0it [00:00, ?it/s]

wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory
wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory
wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory
wandb: WARNING Path wandb_dir/wandb/ wasn't writable, using system temp directory


Training: 0it [00:00, ?it/s]

wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
